In [14]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
!pip install catboost

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20170925-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent

In [15]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from xgboost import XGBRegressor
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import numpy as np

In [16]:
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

import folium

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
train = pd.read_csv('/content/drive/MyDrive/버스/train.csv')
test = pd.read_csv('/content/drive/MyDrive/버스/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/버스/sample_submission.csv')

In [18]:
my_list=list(train['vh_id'].unique()) + list(train['now_longitude'].unique()) + list(train['next_longitude'].unique())
my_dict = {text : i for i, text in enumerate(my_list)}

def transform_df(df:pd.DataFrame):
  df[['vh_id','now_longitude','next_longitude']]=df[['vh_id','now_longitude','next_longitude']].applymap(lambda x:my_dict[x])
    
  def text_to_time(text:str):
    return int(text[:2])

  df['now_arrive_time'] = df['now_arrive_time'].map(text_to_time)

  return df     

train= transform_df(train)
test = transform_df(test)

train = train[train['next_arrive_time']<=1000]

In [19]:
train.loc[(train['now_arrive_time']==5),['time']]='1'
train.loc[(train['now_arrive_time']==6),['time']]='1'

train.loc[(train['now_arrive_time']==7),['time']]='2'
train.loc[(train['now_arrive_time']==8),['time']]='2'
train.loc[(train['now_arrive_time']==9),['time']]='2'

train.loc[(train['now_arrive_time']==10),['time']]='3'
train.loc[(train['now_arrive_time']==11),['time']]='3'
train.loc[(train['now_arrive_time']==12),['time']]='3'
train.loc[(train['now_arrive_time']==12),['time']]='3'
train.loc[(train['now_arrive_time']==13),['time']]='3'
train.loc[(train['now_arrive_time']==14),['time']]='3'
train.loc[(train['now_arrive_time']==15),['time']]='3'

train.loc[(train['now_arrive_time']==16),['time']]='4'
train.loc[(train['now_arrive_time']==17),['time']]='4'
train.loc[(train['now_arrive_time']==18),['time']]='4'
train.loc[(train['now_arrive_time']==19),['time']]='4'

train.loc[(train['now_arrive_time']==20),['time']]='5'
train.loc[(train['now_arrive_time']==21),['time']]='5'
train.loc[(train['now_arrive_time']==22),['time']]='5'
train.loc[(train['now_arrive_time']==23),['time']]='5'
train.loc[(train['now_arrive_time']==0),['time']]='5'

train['time']=train['time'].astype(int)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [20]:
test.loc[(test['now_arrive_time']==5),['time']]='1'
test.loc[(test['now_arrive_time']==6),['time']]='1'

test.loc[(test['now_arrive_time']==7),['time']]='2'
test.loc[(test['now_arrive_time']==8),['time']]='2'
test.loc[(test['now_arrive_time']==9),['time']]='2'

test.loc[(test['now_arrive_time']==10),['time']]='3'
test.loc[(test['now_arrive_time']==11),['time']]='3'
test.loc[(test['now_arrive_time']==12),['time']]='3'
test.loc[(test['now_arrive_time']==12),['time']]='3'
test.loc[(test['now_arrive_time']==13),['time']]='3'
test.loc[(test['now_arrive_time']==14),['time']]='3'
test.loc[(test['now_arrive_time']==15),['time']]='3'

test.loc[(test['now_arrive_time']==16),['time']]='4'
test.loc[(test['now_arrive_time']==17),['time']]='4'
test.loc[(test['now_arrive_time']==18),['time']]='4'
test.loc[(test['now_arrive_time']==19),['time']]='4'

test.loc[(test['now_arrive_time']==20),['time']]='5'
test.loc[(test['now_arrive_time']==21),['time']]='5'
test.loc[(test['now_arrive_time']==22),['time']]='5'
test.loc[(test['now_arrive_time']==23),['time']]='5'
test.loc[(test['now_arrive_time']==0),['time']]='5'

test['time']=test['time'].astype(int)

In [21]:
from sklearn.model_selection import train_test_split

X=train[[ 'vh_id','now_longitude', 'now_arrive_time', 'distance', 'next_longitude', 'time']]
y=train['next_arrive_time']

X_train,X_valid,y_train,y_valid = train_test_split(X,y,test_size=0.1, shuffle=False) 

In [22]:
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor

features = train[[ 'vh_id','now_longitude', 'now_arrive_time', 'distance', 'next_longitude', 'time']]
label = train['next_arrive_time']
A= test[[ 'vh_id','now_longitude', 'now_arrive_time', 'distance', 'next_longitude', 'time']]

kfold  = KFold(n_splits=5)
sub_preds_catboost =np.zeros(A.shape[0])

for train_index, test_index  in kfold.split(features):
    trn_x, val_x  = features.iloc[train_index], features.iloc[test_index]
    trn_y, val_y = label.iloc[train_index], label.iloc[test_index]
    
    cb_model = CatBoostRegressor(task_type='GPU',
                                iterations=3000,
                                 depth=6,
                                 bootstrap_type='Bernoulli',
                                 subsample=0.6,
                                 eval_metric='RMSE'
                                 )
    cat_features = [0,1,4,5]
    cb_model.fit(trn_x, trn_y, eval_set=(val_x, val_y), cat_features=cat_features, verbose=True)

    sub_preds_catboost += cb_model.predict(A) / kfold.n_splits

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
1007:	learn: 32.9337955	test: 33.3482642	best: 33.3472973 (1004)	total: 20.9s	remaining: 41.3s
1008:	learn: 32.9330186	test: 33.3482300	best: 33.3472973 (1004)	total: 20.9s	remaining: 41.3s
1009:	learn: 32.9321434	test: 33.3482899	best: 33.3472973 (1004)	total: 20.9s	remaining: 41.2s
1010:	learn: 32.9273945	test: 33.3479576	best: 33.3472973 (1004)	total: 20.9s	remaining: 41.2s
1011:	learn: 32.9263993	test: 33.3473971	best: 33.3472973 (1004)	total: 21s	remaining: 41.2s
1012:	learn: 32.9257291	test: 33.3469507	best: 33.3469507 (1012)	total: 21s	remaining: 41.2s
1013:	learn: 32.9251672	test: 33.3469550	best: 33.3469507 (1012)	total: 21s	remaining: 41.2s
1014:	learn: 32.9246558	test: 33.3473800	best: 33.3469507 (1012)	total: 21.1s	remaining: 41.2s
1015:	learn: 32.9238555	test: 33.3473629	best: 33.3469507 (1012)	total: 21.1s	remaining: 41.2s
1016:	learn: 32.9208680	test: 33.3466070	best: 33.3466070 (1016)	total: 21.1s	remaining: 41.1s
1017:	learn: 32.9198

In [23]:
sub_preds_catboost

array([ 35.55758482,  46.88010423, 132.78729283, ..., 115.70306731,
        29.70768457,  94.15952601])

In [24]:
sub_preds_catboost
submission[target]=sub_preds_catboost
submission.to_csv('KFold.csv',index=False)